In [ ]:
import nanopyx.opencl as npx_cl

npx_cl.print_opencl_info()

In [ ]:
import pyopencl as cl
import pyopencl.array as pycl_array  # Import PyOpenCL Array (a Numpy array plus an OpenCL buffer object)
import numpy as np
import numpy.linalg as la
import datetime
from time import time

a = np.random.rand(1000).astype(np.float32)
b = np.random.rand(1000).astype(np.float32)
c_result = np.empty_like(a)

for platform in cl.get_platforms():
    for device in platform.get_devices():
        print("===============================================================")
        print("Platform name:", platform.name)
        print("Platform profile:", platform.profile)
        print("Platform vendor:", platform.vendor)
        print("Platform version:", platform.version)
        print("---------------------------------------------------------------")
        print("Device name:", device.name)
        print("Device type:", cl.device_type.to_string(device.type))
        print("Device memory: ", device.global_mem_size//1024//1024, 'MB')
        print("Device max clock speed:", device.max_clock_frequency, 'MHz')
        print("Device compute units:", device.max_compute_units)

        # Simnple speed test
        ctx = cl.Context([device])
        queue = cl.CommandQueue(ctx, 
                properties=cl.command_queue_properties.PROFILING_ENABLE)

        mf = cl.mem_flags

        a = pycl_array.to_device(queue, np.arange(50000, dtype=np.float32))
        b = pycl_array.to_device(queue, np.arange(50000, dtype=np.float32))  
        # Create two random pyopencl arrays
        c = pycl_array.empty_like(a)  # Create an empty pyopencl destination array

        program = cl.Program(ctx, """
        __kernel void sum(__global const float *a, __global const float *b, __global float *c)
        {
        int i = get_global_id(0);
        c[i] = a[i] + b[i];
        }""").build()  # Create the OpenCL program

        exec_evt = program.sum(queue, a.shape, None, a.data, b.data, c.data)  # Enqueue the program for execution and store the result in c
        exec_evt.wait()
        elapsed = 1e-9*(exec_evt.profile.end - exec_evt.profile.start)

        print(c.get())

        print("Execution time of test: %g s" % elapsed)


In [ ]:
import nanopyx.opencl as npx_cl
import nanopyx.opencl._cl_mandelbrot_benchmark as npx_cl_mandelbrot
import matplotlib.pyplot as plt

im = npx_cl_mandelbrot.mandelbrot(1000, 1000, 4)
plt.imshow(im)